In [ ]:
from __future__ import annotations

import math

import ipywidgets as widgets
import numpy as np
import rerun as rr
import rerun.blueprint as rrb
from rerun.notebook import Viewer, ViewerCallbacks
from rerun.utilities import build_color_grid

In [ ]:
# To show something interesting, we first need to log some data.
rr.init("rerun_example_callbacks")

# This will output a cube grid of points. We'll be able to select
# either the entire grid, or individual points.
STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time("step", sequence=t)
    cube = build_color_grid(10, 10, 10, twist=twists[t])
    rr.log("cube", rr.Points3D(cube.positions, colors=cube.colors, radii=0.5))

rr.send_blueprint(rrb.Blueprint(rrb.Spatial3DView(name="Everything")))


# To register callbacks, first inherit from the `ViewerCallbacks` base class,
# and override the event methods you are interested in.
class Example(ViewerCallbacks):
    def __init__(self) -> None:
        # Because this is a class, we can store whatever state we need in it.
        self.selection = ""
        self.time = ""
        self.timeline = ""

        # Create a Viewer instance, and register ourselves to it.
        # The same callbacks may be registered to multiple Viewers at a time,
        # but there is currently no way to know which Viewer the events are coming from.
        # A viewer may also have more than one set of callbacks registered at a time.
        self.viewer = Viewer()
        self.viewer.register_callbacks(self)

        # We'll display the event payloads in basic labels.
        self.time_label = widgets.Label()
        self.selection_label = widgets.Label()
        self.label = widgets.VBox([self.time_label, self.selection_label])

    def _ipython_display_(self):
        # This is called when the object is the last value in a cell.
        # We're supposed to call our inner widgets' `display` functions as side effects.
        self.viewer.display()
        display(self.label)

    def update_label(self) -> None:
        self.time_label.value = f"time={self.timeline} @ {self.time}"
        self.selection_label.value = f"selected=[{self.selection}]"

    def on_selection_change(self, selection) -> None:
        # Because it is possible to select multiple items at a time in the Viewer,
        # the selection change event yields a list of items.
        label = []
        for item in selection:
            if item.kind == "entity":
                # Entities always include their entity path,
                # and optionally also:
                # * The instance ID
                # * The mouse coordinates in _scene space_ at the time of the selection
                #   in a 2D or 3D view. Scene space here means in the coordinate system
                #   of the view and relative to its origin.
                # * The view name
                entity_path, instance_id, view_name, position = (
                    item.entity_path,
                    item.instance_id,
                    item.view_name,
                    item.position,
                )
                label.append(f"Entity({entity_path=}, {instance_id=}, {view_name=}, {position=})")
            elif item.kind == "container":
                # Containers and Views only include their ID.
                container_id, container_name = item.container_id, item.container_name
                label.append(f"Container({container_id=}, {container_name=})")
            elif item.kind == "view":
                view_id, view_name = item.view_id, item.view_name
                label.append(f"View({view_id=}, {view_name=})")
        self.selection = ", ".join(label)
        self.update_label()

    def on_time_update(self, time) -> None:
        # We'll receive a new time each time the Viewer updates it.
        # This may be as frequently as it can render in case it is being played back,
        # or on each timeline click in case the user seeks to some time point.

        self.time = str(time)
        self.update_label()

    def on_timeline_change(self, timeline, time) -> None:
        # The current time is stored per timeline, so we get both the timeline name
        # and the time here.
        self.timeline = str(timeline)
        self.time = str(time)
        self.update_label()


# Start the viewer instance
Example()